In [62]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date


loading in data 


In [63]:
os.getcwd()


'c:\\Users\\Olivia_Pinney\\Documents\\UChicago_Materials\\Data_Viz\\CAPP30239_FA22\\final\\data'

# Department Roster

In [64]:
history_roster=pd.read_csv("extra_data/roster_1936-2017_2017-04.csv")
# history_unit=pd.read_csv(data_path_iigh["history_unit"]) 

In [65]:
df=history_roster 
df["resignation_date"]=df["resignation_date"].fillna("2999-01-01")
df["resignation_year"]=df["resignation_date"].apply(lambda x: int(str(x)[0:4]))

df["appointed_date"]=df["appointed_date"].fillna("1000-01-01")
df["appointed_year"]=df["appointed_date"].apply(lambda x: int(str(x)[0:4]))

df=df[df["appointed_year"]<2017] #officer in 2016
df=df[df["resignation_year"]>2015] #officer in 2016

#check that no one is duplicated
temp=df.UID.value_counts().reset_index()
problem_uids=temp[temp.UID>1]["index"]
print(problem_uids)
print(df.shape)
df=df[~(df["UID"].isin(problem_uids) & ~df["current_status"]>0)]
print(df.shape)


0    131785.0
1    131788.0
2    131787.0
3    131783.0
4    131784.0
5    131789.0
6    131786.0
Name: index, dtype: float64
(13126, 36)
(13119, 36)


In [66]:
max(history_roster.appointed_year)

2017

In [67]:
df["unit_cat"]=np.where(df["unit_description"].str.contains("DISTRICT 0"),"DISTRICT",
    np.where(df["unit_description"].str.contains("DETECTIVE"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("SPECIAL INVESTIGATIONS UNIT"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("TRAFFIC"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSIT"),"TRANSPORT",    
    np.where(df["unit_description"].str.contains("AIRPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("GANG ENFORCEMENT"),"GANG ENFORCEMENT",
    "OTHER"
    ))))))))

In [68]:
temp=df[["unit_description","unit_cat"]].value_counts(sort=True)
temp=temp.reset_index()
#temp=temp.set_index("unit_description")
temp.columns=["unit_description","unit_cat","headcount"]

temp.to_json("departments_basic.json",orient="records")


# Section of Complaints

In [69]:
#merge complaints
complaints=pd.read_csv("extra_data/complaints-accused.csv")
complaints_detailed=pd.read_csv("extra_data/complaints-complaints.csv")
complaint_types=pd.read_csv("extra_data/Complaint Categories.csv")

cmp= complaints.merge(complaints_detailed[["cr_id", "complaint_date"]],on="cr_id",how="left")
cmp["code"]=cmp["complaint_code"]
complaint_types=complaint_types[~pd.isna(complaint_types["111"])]
complaint_types["code"]=complaint_types["111"]
cmp= cmp.merge(complaint_types,on="code",how="left")

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_10608\2543640591.py:2: DtypeWarning: Columns (3,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints=pd.read_csv("extra_data/complaints-accused.csv")
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_10608\2543640591.py:3: DtypeWarning: Columns (0,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_detailed=pd.read_csv("extra_data/complaints-complaints.csv")


Need to add a section on the outcome of the complaints

In [70]:
#complaint results
comp_cat={"NAF":"No Affidavit", "EX":"Complaint Dismissed", "UN":"Complaint Dismissed","NS":"Insufficient Evidence","SU":"Complaint Accepted", "DIS":"Complaint Accepted","NC":"Missing Complaint Type","Missing":"Missing"}
#NC is missing

#outcome
#complaints.final_outcome.value_counts()

#less important:
dept_exclude=["OPERATION/PERSONNEL VIOLATION:  MISCELLANEOUS","MISCELLANEOUS", "OPERATION/PERSONNEL VIOLATION:  REPORTS - FAIL TO SUBMIT/IMPROPER", "OPERATION/PERSONNEL VIOLATION:  INVENTORY PROCEDURES","REPORTS - FAILED TO SUBMIT/IMPROPER","INVENTORY PROCEDURES","OPERATION/PERSONNEL VIOLATION: COMMUNICATION OPERATION PROCEDURES","COMMUNICATION OPERATIONS PROCEDURES","OPERATION/PERSONNEL VIOLATION:  VEHICLE LICENSING - CITY","MISUSE OF DEPARTMENT EQUIPMENT/SUPPLIES","INSUBORDINATION","OPERATION/PERSONNEL VIOLATION:  INSUBORDINATION","MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  WEAPON/AMMUN./UNIFORM DEVIATION","FAIL TO OBTAIN A COMPLAINT REGISTER NUMBER","OPERATION/PERSONNEL VIOLATION: MISUSE OF DEPT. EQUIPMENT/SUPPLIES","OPERATION/PERSONNEL VIOLATION:  ABSENT WITHOUT PERMISSION","WEAPON/AMMUNITION/UNIFORM DEVIATION","SUPERVISOR RESPONSIBILITY:  FAIL TO OBTAIN COMPLAINT REGISTER NO.","SUPERVISOR RESPONSIBILITY:  MISCELLANEOUS","OPERATION/PERSONNEL VIOLATION:  LEAVING ASGMT (DIST/BEAT/SECT/CT)","ABSENT WITHOUT PERMISSION","OPERATION/PERSONNEL VIOLATION:  RESIDENCY","LEAVING ASSIGNMENT (DISTRICT, BEAT, SECTOR, COURT)","RESIDENCY","OPERATION/PERSONNEL VIOLATION:  COMPENSATORY TIME","VEHICLE LICENSING - CITY","OPERATION/PERSONNEL VIOLATION:  LATE FOR ROLL CALL/ASSIGNMENT/CT.","COMPENSATORY TIME","SUPVR. RESPONSIBILITY:  PROPER ACTION REVIEW/INSPECT- SUBORDINATE","OPERATION/PERSONNEL VIOLATION:  LUNCH/PERSONAL","LATE - ROLL CALL/ASSIGNMENT/COURT","LUNCH/PERSONAL VIOLATIONS","PROPER ACTION REVIEW/INSPECT - SUBORDINATE","EQUIPMENT IRREGULARITY"]


In [71]:
force=["EXCESSIVE FORCE","STRIKE","ASSAULT","BATTERY","PUSH","PULL","GRAB","CHOKED","KICKED","DRAGGED","TASER","SHOTS FIRED","COERCION - VIOLENCE","MURDER","MANSLAUGHTER","TAKE DOWN","INJURY","DEATH","FIREARM","WEAPON","PHYSICAL CONTACT"]
detain=["DETENTION","WARRANT","SEARCH","ARRESTEE","ARREST","LOCKUP","BONDING","BOOKING","PROCESSING","PRISONER'S PROPERTY", "PRISONERS PROPERTY"]
hate=["RACIAL","SEXUAL ORIENTATION","VERBAL ABUSE", "USE OF PROFANITY", "THREATS"]
crime=["COMMISSION OF CRIME", "DAMAGE","TRESPASSING", "BURGLARY", "THEFT", "FELONY","DRUGS","CONTR. SUB", "ROBBERY", "SHOPLIFTING","DOMESTIC","RAPE"]

#contains
citizen_cats={"EXCESSIVE FORCE":"EXCESSIVE FORCE","STRIKE":"EXCESSIVE FORCE","ASSAULT":"EXCESSIVE FORCE","BATTERY":"EXCESSIVE FORCE","PUSH":"EXCESSIVE FORCE","PULL":"EXCESSIVE FORCE","GRAB":"EXCESSIVE FORCE","INJURY":"EXCESSIVE FORCE","CHOKED":"EXCESSIVE FORCE","KICKED":"EXCESSIVE FORCE","DRAGGED":"EXCESSIVE FORCE","TASER":"EXCESSIVE FORCE","SHOTS FIRED":"EXCESSIVE FORCE","COERCION - VIOLENCE":"EXCESSIVE FORCE","MURDER/MANSLAUGHTER, ETC.":"EXCESSIVE FORCE","TAKE DOWN":"EXCESSIVE FORCE",
    "INJURY/DEATH":"INJURY/DEATH", 
    "UNNECESSARY PHYSICAL CONTACT":"UNNECESSARY PHYSICAL CONTACT",
    "FIREARM": "WEAPON USAGE", "WEAPON":"WEAPON USAGE",
    "EXCESSIVE DETENTION":"EXCESSIVE DETENTION",
    "WARRANT": "WARRANT / SEARCH VIOLATION","SEARCH":"SEARCH WITHOUT WARRANT",
    "ARRESTEE": "HANDLING OF ARREST / BOOKING", "ARREST/LOCKUP": "HANDLING OF ARREST / BOOKING", "BONDING/BOOKING/PROCESSING": "HANDLING OF ARREST / BOOKING","PRISONER'S PROPERTY":"HANDLING OF ARREST / BOOKING" , "PRISONERS PROPERTY":"HANDLING OF ARREST / BOOKING",
    "ILLEGAL ARREST": "ILLEGAL ARREST",
    "IMPROPER DETENTION":"UNLAWFUL DETENTION", "UNLAWFUL / EXCESSIVE INVESTIGATIVE DETENTION (WITNESS)": "UNLAWFUL DETENTION",
    "RACIAL": "RACIAL PROFILING / ABUSIVE LANGUAGE",
    "SEXUAL ORIENTATION":"SEXUAL ORIENTATION",
    "VERBAL ABUSE":"VERBAL ABUSE",  "USE OF PROFANITY": "VERBAL ABUSE", "THREATS": "VERBAL ABUSE",
    "DOMESTIC":"DOMESTIC ALTERCATION",
    "RAPE/SEX":"RAPE/SEX OFFENSES",
    "TRAFFIC": "TRAFFIC (NOT BRIBERY/EX-FORCE)",
    "NO ARREST": "NO ARREST",
    "COMMISSION OF CRIME":"CRIME", "DAMAGE/TRESPASSING PROPERTY":"CRIME", "BURGLARY":"CRIME", "THEFT":"CRIME", "OTHER FELONY":"CRIME","DRUGS/CONTR. SUB., POSSESSION OR SALE":"CRIME", "ROBBERY":"CRIME", "SHOPLIFTING":"CRIME",
    "ATTORNEY/RELATIVE PRIVILEGES":"COMMUNICATION","FIRST AMENDMENT":"COMMUNICATION",
    "MISCONDUCT DURING ISSUANCE OF CITATION":"OTHER","CIVIL SUIT":"OTHER", "IMPROPER PROCESSING/REPORTING/PROCEDURES":"OTHER","ESCAPE":"OTHER","MISCELLANEOUS":"OTHER","PARKING COMPLAINTS":"OTHER","VIOLATION (OTHER THAN D.U.I.) - ON DUTY":"OTHER","POLICE IMPERSONATOR - ADV SECTION USE ON CLOSING ONLY":"OTHER"}


In [72]:
cmp["civ_complaint_force"]=False
cmp["civ_complaint_detain"]=False
cmp["civ_complaint_hate"]=False
cmp["civ_complaint_crime"]=False

cmp["civ_complaint_count"]=np.where(cmp["CITIZEN / DEPT"]=="CITIZEN",1,0)
cmp["civ_complaint_count_desc"]=np.where(cmp["civ_complaint_count"],cmp.complaint_category,"")
cmp["civ_complaint_count_desc"]=np.where(cmp["civ_complaint_count_desc"].isna(),"Missing",cmp["civ_complaint_count_desc"])

for word in force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_force"])
for word in detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_detain"])
for word in hate:
    cmp["civ_complaint_hate"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_hate"])
for word in crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_crime"])

duplicate_words_detain=["EXCESSIVE FORCE:  ARRESTEE","EXCESSIVE FORCE:  NO ARREST"]
duplicate_words_force=["COMMISSION OF CRIME:  ASSAULT/BATTERY","COMMISSION OF CRIME:  MURDER/MANSLAUGHTER ETC","ARREST/LOCKUP PROCEDURE:  PROPER CARE/INJURY/DEATH"]
duplicate_words_crime=["EXCESSIVE FORCE:  DOMESTIC"]

#identify duplicate words here 
#cmp[["civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]].value_counts()
#cmp.loc[(cmp["civ_complaint_force"]==True) & (cmp["civ_complaint_detain"]==True)]["civ_complaint_count_desc"].value_counts()

for word in duplicate_words_force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_force"])
for word in duplicate_words_detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_detain"])
for word in duplicate_words_crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_crime"])


In [73]:
cmp["comp_cat"]=np.where(cmp.final_finding.isna(),"Missing",cmp.final_finding)
cmp["comp_cat"]=cmp["comp_cat"].replace(comp_cat)

cmp["dept_complaint_count"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",True,False)
cmp["dept_complaint_cleaned_count"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",np.where(cmp.complaint_category.isin(dept_exclude),False,True),False)
cmp["dept_complaint_cleaned_count_desc"]=np.where(cmp["dept_complaint_cleaned_count"],cmp.complaint_category,"")

cmp["civ_complaint_cat"]=np.where(cmp.civ_complaint_force,"FORCE",
    np.where(cmp.civ_complaint_detain,"DETAIN",
    np.where(cmp.civ_complaint_hate,"HATE",
    np.where(cmp.civ_complaint_crime,"CRIME",
    np.where(cmp.civ_complaint_count,"OTHER","")))))
    
#checking count made correctly
#cmp[cmp.dept_complaint_count==1][["complaint_category","dept_complaint_cleaned_count"]].value_counts()

In [74]:
cmp_counts=cmp.groupby("UID",as_index=False).agg({"dept_complaint_count":"sum","dept_complaint_cleaned_count":"sum","civ_complaint_count":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum"})
df=df.merge(cmp_counts,on="UID",how="left")

counts=["dept_complaint_cleaned_count","civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]
col_names=["dept_complaint_cleaned_count_desc","civ_complaint_count_desc","civ_complaint_force_desc","civ_complaint_detain_desc","civ_complaint_hate_desc","civ_complaint_crime_desc"]
for i, count in enumerate(counts):
    print(count)
    temp=cmp[cmp[count]==True].groupby("UID")["complaint_category"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")


dept_complaint_cleaned_count
civ_complaint_count
civ_complaint_force
civ_complaint_detain
civ_complaint_hate
civ_complaint_crime


## TTR

In [75]:
#merge trr
trr_overall=pd.read_csv("extra_data/TRR-main_2004-2016_2016-09.csv")
trr_subjects=pd.read_csv("extra_data/TRR-subjects_2004-2016_2016-09.csv")
trr_officers=pd.read_csv("extra_data/TRR-officers_2004-2016_2016-09.csv")
trr_actions=pd.read_csv("extra_data/TRR-actions-responses_2004-2016_2016-09.csv")

trr=trr_overall.merge(trr_officers,on="trr_id",suffixes=("","_officer")) #,how="left"
trr=trr.merge(trr_subjects,on="trr_id",suffixes=("","_subject")) #,how="left"
trr=trr.merge(trr_actions[trr_actions.person=="Member Action"],on="trr_id",suffixes=("","_subject")) #,how="left"


trr["trr_year"]=trr["trr_date"].apply(lambda x: int(str(x)[0:4]))
trr_17=trr #[trr["trr_year"]==2017]


In [76]:
action_cat={"Missing":"Missing","Verbal Commands":"Other","Member Presence":"Other","Physical Force - Stunning":"Use of Force","Physical Force - Holding":"Use of Force","Physical Force - Direct Mechanical":"Use of Force","Other Force":"Use of Force","Taser":"Use of Weapon","Chemical":"Use of Weapon","Impact Weapon":"Use of Weapon","Firearm":"Use of Weapon","Taser Display":"Other","Chemical (Authorized)":"Use of Weapon","Other":"Other"} 

In [77]:
trr_17["force_cat"]=np.where(trr_17.force_type.isna(),"Missing",trr_17.force_type.replace(action_cat))
trr_17["count"]=True
trr_17["weapon_count"]=np.where(trr_17["force_cat"]=="Use of Weapon",True,False)
trr_17["force_count"]=np.where(trr_17["force_cat"]=="Use of Force",True,trr_17["weapon_count"])
trr_17["trr_count"]=np.where(trr_17["force_cat"]=="Other",True,trr_17["force_count"])



In [78]:
trr_17_counts=trr_17.groupby("UID",as_index=False).agg({"force_count":"sum","weapon_count":"sum","trr_count":"sum"})
    
df=df.merge(trr_17_counts,on="UID",how="left")

counts=["force_count","weapon_count","trr_count"]
col_names=["force_count_desc","weapon_count_desc","trr_count_desc"]
for i, count in enumerate(counts):
    temp=trr_17[trr_17[count]==True].groupby("UID")["action"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")


In [79]:
temp

,UID,trr_count_desc
0,100007.0,"[MEMBER PRESENCE, VERBAL COMMANDS, TAKE DOWN/E..."
1,100009.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, MEMBER PRESE..."
2,100010.0,"[TASER (PROBE DISCHARGE), TASER (PROBE DISCHAR..."
3,100011.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, CLOSED HAND ..."
4,100012.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, TAKE DOWN/EM..."
...,...,...
10772,132446.0,"[O.C./CHEMICAL WEAPON, ESCORT HOLDS, MEMBER PR..."
10773,132453.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, MEMBER PRESE..."
10774,132466.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, VERBAL COMMA..."
10775,132518.0,"[TAKE DOWN/EMERGENCY HANDCUFFING, MEMBER PRESE..."


# Export

In [80]:
col_names=["force_count","weapon_count","trr_count","dept_complaint_count","dept_complaint_cleaned_count","civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]

for col in col_names:
    df[col+"_persons_with"]=np.where(df[col]>0,1,0)

In [81]:
unit_all=df.groupby("unit_description").agg({"UID":"count","force_count":"sum","weapon_count":"sum","trr_count":"sum","dept_complaint_count":"sum","dept_complaint_cleaned_count":"sum","civ_complaint_count":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum","force_count_persons_with":"sum","weapon_count_persons_with":"sum","trr_count_persons_with":"sum","dept_complaint_count_persons_with":"sum","dept_complaint_cleaned_count_persons_with":"sum","civ_complaint_count_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"}).reset_index()
unit_cats=df.groupby("unit_cat").agg({"UID":"count","force_count":"sum","weapon_count":"sum","trr_count":"sum","dept_complaint_count":"sum","dept_complaint_cleaned_count":"sum","civ_complaint_count":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum","force_count_persons_with":"sum","weapon_count_persons_with":"sum","trr_count_persons_with":"sum","dept_complaint_count_persons_with":"sum","dept_complaint_cleaned_count_persons_with":"sum","civ_complaint_count_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"}).reset_index()

unit_all.to_json("departments.json",orient="records")
unit_cats.to_json("departments_cats.json",orient="records")

In [82]:
df['appointed_year']=df['appointed_date'].apply(lambda x: str(x)[0:4])
df['appointed_year']=df['appointed_year'].astype("int")
df["years_in_2016"]=2017-df['appointed_year'].astype(int)
df["years_in_2016"]=np.where(df["years_in_2016"]==1017,-1,df["years_in_2016"])
df['force_count_per_year']=df['force_count']/df["years_in_2016"]
df['dept_complaint_cleaned_per_year']=df['dept_complaint_cleaned_count']/df["years_in_2016"]
df['civ_complaint_per_year']=df['civ_complaint_count']/df["years_in_2016"]



df["full_name"]=df['first_name']+" "
df["full_name"]=np.where(df["middle_initial"].isna(),df["full_name"],df["full_name"]+df['middle_initial']+". ")
df["full_name"]=np.where(df["middle_initial2"].isna(),df["full_name"],df["full_name"]+df['middle_initial2']+". ")
df["full_name"]=df["full_name"]+df['last_name']
df["full_name"]=np.where(df["suffix_name"].isna(),df["full_name"],df["full_name"]+" "+df['suffix_name'])



In [83]:
df['dept_complaint_cleaned_per_year']

0        0.166667
1        0.000000
2        0.090909
3        0.000000
4        0.333333
           ...   
13114    0.166667
13115         NaN
13116    0.266667
13117    0.000000
13118         NaN
Name: dept_complaint_cleaned_per_year, Length: 13119, dtype: float64

In [85]:
cols_to_keep=['gender', 'race', 'birth_year', 'current_age','appointed_date', 'current_rank', 'unit_description', 'first_name', 'last_name', 'middle_initial', 'suffix_name','UID','dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'dept_complaint_desc', 'civ_complaint_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_count','weapon_count','trr_count', 'action_force_desc', 'action_weapon_desc']
cols_to_keep=['full_name','last_name','birth_year','appointed_date', 'current_rank', 'unit_description','dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'dept_complaint_cleaned_count_desc', 'civ_complaint_count_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_count','weapon_count','trr_count', 'force_count_desc', 'weapon_count_desc','force_count_per_year','dept_complaint_cleaned_per_year','civ_complaint_per_year','years_in_2016']

cols_to_keep_int=['dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'force_count','weapon_count','trr_count','force_count_per_year','dept_complaint_cleaned_per_year','civ_complaint_per_year','years_in_2016']
cols_to_keep_str=['full_name','last_name','birth_year','appointed_date', 'current_rank', 'unit_description']
cols_to_keep_list=['dept_complaint_cleaned_count_desc', 'civ_complaint_count_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'action_force_desc', 'action_weapon_desc']

for col in cols_to_keep_int:
    df[col]=np.where(df[col].isna(),0,df[col])
for col in cols_to_keep_str:
    df[col]=np.where(df[col].isna(),"",df[col])
# for col in cols_to_keep_list:
#     df[col]=np.where(df[col].isna(),list(),df[col])


df[cols_to_keep].to_json("officers_2016.json",orient="records")


# Update Department Json

In [86]:
import json

with open("departments.json") as file:
    data = json.load(file)

with open("officers_2016.json") as file:
    officers = json.load(file)

for d in data:
    d["officers"]=[]
    temp=[]
    for officer in officers:
        if officer["unit_description"]==d["unit_description"]:
            d["officers"].append(officer)

with open("departments_officers.json", "w") as outfile:
    json.dump(data, outfile)


In [ ]:
# temp=df[df.weapon_count>10].action_force_desc
# temp[12331]
# #df.columns

#df.force_count.value_counts()
cmp.dept_complaint_desc.value_counts()

                                                                     200812
INADEQUATE/FAILURE TO PROVIDE SERVICE                                 15134
OPERATION/PERSONNEL VIOLS.:  NEGLECT OF DUTY / CONDUCT UNBECOMING      8517
NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY                           8060
OPERATION/PERSONNEL VIOLATION: INADEQUATE/FAIL TO PROVIDE SERVICE      7269
FEDERAL CIVIL SUIT                                                     1781
STATE CIVIL SUIT                                                       1084
SEXUAL HARRASSMENT                                                      271
OPERATION/PERSONNEL VIOLATION:  COURT IRREGULARITIES                    222
MISUSE DEPARTMENT RECORDS                                               202
OPERATION/PERSONNEL VIOLATION:  SEXUAL HARRASEMENT                      142
COURT IRREGULARITIES                                                    134
OPERATION/PERSONNEL VIOLATION:  SECONDARY/SPECIAL EMPLOYMENT            117
PROPER ACTIO